In [4]:
from __future__ import print_function
from __future__ import print_function
from __future__ import division
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torchvision
import matplotlib.pyplot as plt
import csv
from torchvision import models
import time
import os
import copy
import cv2
import numpy as np
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import scipy.io
import numpy as np
from pathlib import Path
import glob

import torch.utils.data as data
import pandas as pd
from PIL import Image
import os.path

PyTorch Version:  2.1.2+cu121
Torchvision Version:  0.16.2+cu121


In [5]:
IMG_EXTENSIONS = [
   '.jpg', '.JPG', '.jpeg', '.JPEG',
   '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP','.mat',
]


def is_image_file(filename):
   return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def find_classes(dir):
   classes = os.listdir(dir)
   classes.sort()
   class_to_idx = {classes[i]: i for i in range(len(classes))}
   return classes, class_to_idx


def make_dataset(dir, class_to_idx):
   images = []
   for target in os.listdir(dir):
       d = os.path.join(dir, target)
       if not os.path.isdir(d):
           continue

       for filename in os.listdir(d):
           if is_image_file(filename):
               path = '{0}/{1}'.format(target, filename)
               #print(path)
               item = (path, class_to_idx[target])
               images.append(item)

   return images

def default_loader(path):
   return Image.open(path).convert('RGB')

def mat_loader(path):
   return scipy.io.loadmat(path1)

In [6]:
classes1, class_to_idx1 = find_classes("/home/user1/icip/final_patch_rafi_random/train/")
       
imgs1 = make_dataset("/home/user1/icip/final_patch_rafi_random/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

print("="*100)

imgs1 = make_dataset("/home/user1/icip/final_patch_rafi_similar/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

print("="*100)

imgs1 = make_dataset("/home/user1/icip/final_patch_merged_rafi/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

{'D01_Samsung_Galaxy_S20Plus': 0, 'D02_Nothing_One': 1, 'D03_Samsung_Galaxy_A03': 2, 'D04_Samsung_Galaxy_M04': 3, 'D05_Vivo_V9_Pro': 4, 'D06_Apple_Iphone_12Mini': 5, 'D07_Apple_Iphone_11': 6, 'D08_Redmi_Note_8Pro': 7, 'D09_Samsung_Galaxy_J8_10G': 8, 'D10_Samsung_Galaxy_F41': 9, 'D11_OnePlus_8T': 10, 'D12_Vivo_Y02t': 11, 'D13_Oppo_A17k': 12, 'D14_Samsung_Galaxy_S20FE': 13, 'D15_Motorola_Motog60': 14, 'D16_Samsung_Galaxy_S21FE': 15, 'D17_Apple_Iphone_12': 16, 'D18_IQOO_Z3': 17, 'D19_IQOO_Z6_Lite': 18, 'D20_Motorola_MotoG73_5G': 19, 'D21_OnePlus_10Pro_5G': 20, 'D22_Poco_F5': 21, 'D23_Poco_F5_Pro_5G': 22, 'D24_Realme_8': 23, 'D25_Realme_X3_Superzoom': 24, 'D26_Redmi_9i_Sport': 25, 'D27_Redmi_Note10_Pro': 26, 'D28_Apple_Iphone_13': 27, 'D29_Apple_Iphone_15': 28, 'D30_Vivo_Y75': 29} 342720
{'D01_Samsung_Galaxy_S20Plus': 0, 'D02_Nothing_One': 1, 'D03_Samsung_Galaxy_A03': 2, 'D04_Samsung_Galaxy_M04': 3, 'D05_Vivo_V9_Pro': 4, 'D06_Apple_Iphone_12Mini': 5, 'D07_Apple_Iphone_11': 6, 'D08_Redmi_No

In [7]:
class ImageFolderLoader(data.Dataset):
   def __init__(self, root1,transform_1=None,
                target_transform=None,
                loader=default_loader):
       classes1, class_to_idx1 = find_classes(root1)
       
       imgs1 = make_dataset(root1, class_to_idx1)
      

       self.root1 = root1
       self.imgs1 = imgs1
       self.classes1 = classes1
       self.class_to_idx1 = class_to_idx1
       self.target_transform = target_transform
       self.loader = loader
       self.img_transform = transform_1
        
       
       

   def __getitem__(self, index):
    

       path1, target1 = self.imgs1[index]
       filename = Path(path1).stem 
    
       img1 = self.loader(os.path.join(self.root1, path1))  
       
       if self.img_transform is not None:
           img1 = self.img_transform(img1)
        
       img1 = np.array(img1) 
       img1 = torch.from_numpy((img1)).float() 

        
       if self.target_transform is not None:
           target1 = self.target_transform(target)
        
       target1 = torch.eye(30)[target1]      
            
       #print(img1.shape,img_mat.shape)      

       return img1,target1,filename

   def __len__(self):
       return len(self.imgs1)

In [8]:
data_transforms = transforms.Compose([
transforms.ToTensor()
])


batchsize=1

val_dataset_random = ImageFolderLoader(
        "/home/user1/icip/final_patch_rafi_random/test/",
        transform_1=data_transforms
    )

test_loader_random = torch.utils.data.DataLoader(
        val_dataset_random, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

val_dataset_similar = ImageFolderLoader(
        "/home/user1/icip/final_patch_rafi_similar/test/",
        transform_1=data_transforms
    )

test_loader_similar = torch.utils.data.DataLoader(
        val_dataset_similar, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

val_dataset_merged = ImageFolderLoader(
        "/home/user1/icip/final_patch_merged_rafi/test/",
        transform_1=data_transforms
    )

test_loader_merged = torch.utils.data.DataLoader(
        val_dataset_merged, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

In [9]:
len(val_dataset_random), len(val_dataset_similar), len(val_dataset_merged)

(342720, 307200, 649920)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.rem1_conv1 = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=(3,3),stride=1,padding=1)
        self.rem1_bn1 = nn.BatchNorm2d(64)
        self.rem1_conv2 = nn.Conv2d(in_channels=67,out_channels=64,kernel_size=(3,3),stride=1,padding=1)
        self.rem1_bn2 = nn.BatchNorm2d(64)
        self.rem1_conv3 = nn.Conv2d(in_channels=67,out_channels=3,kernel_size=(3,3),stride=1,padding=1)
        self.rem1_bn3 = nn.BatchNorm2d(3)
        
        
        self.rem2_conv1 = nn.Conv2d(in_channels=3,out_channels=128,kernel_size=(3,3),stride=1,padding=1)
        self.rem2_bn1 = nn.BatchNorm2d(128)
        self.rem2_conv2 = nn.Conv2d(in_channels=131,out_channels=128,kernel_size=(3,3),stride=1,padding=1)
        self.rem2_bn2 = nn.BatchNorm2d(128)
        self.rem2_conv3 = nn.Conv2d(in_channels=131,out_channels=3,kernel_size=(3,3),stride=1,padding=1)
        self.rem2_bn3 = nn.BatchNorm2d(3)
        
        self.rem3_conv1 = nn.Conv2d(in_channels=3,out_channels=256,kernel_size=(3,3),stride=1,padding=1)
        self.rem3_bn1 = nn.BatchNorm2d(256)
        self.rem3_conv2 = nn.Conv2d(in_channels=259,out_channels=256,kernel_size=(3,3),stride=1,padding=1)
        self.rem3_bn2 = nn.BatchNorm2d(256)
        self.rem3_conv3 = nn.Conv2d(in_channels=259,out_channels=3,kernel_size=(3,3),stride=1,padding=1)
        self.rem3_bn3 = nn.BatchNorm2d(3)
        
        
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=(7,7),stride=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.prelu1 = nn.PReLU()
        
        self.conv2 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=(5,5),stride=2)
        self.bn2 = nn.BatchNorm2d(128)
        self.prelu2 = nn.PReLU()
        
        self.conv3 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=(3,3),stride=2)
        self.bn3 = nn.BatchNorm2d(256)
        self.prelu3 = nn.PReLU()
        
        self.conv4 = nn.Conv2d(in_channels=256,out_channels=512,kernel_size=(2,2),stride=2,padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.prelu4 = nn.PReLU()
        
   
        self.avgpool1= nn.AvgPool2d(kernel_size=(4,4),stride=2)
        
        self.conv5 = nn.Conv2d(in_channels=512,out_channels=30,kernel_size=(1,1),stride=1)
        
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x1):
        
        x = x1
        x = self.rem1_conv1(x)
        x = self.rem1_bn1(x)
        x = torch.cat((x,x1),dim=1)
        x = self.rem1_conv2(x)
        x = self.rem1_bn2(x)
        x = torch.cat((x,x1),dim=1)
        x = self.rem1_conv3(x)
        x = self.rem1_bn3(x)
        
        x = x1-x
        x1= x
        
    
        x = self.rem2_conv1(x)
        x = self.rem2_bn1(x)
        x = torch.cat((x,x1),dim=1)
        x = self.rem2_conv2(x)
        x = self.rem2_bn2(x)
        x = torch.cat((x,x1),dim=1)
        x = self.rem2_conv3(x)
        x = self.rem2_bn3(x)
        
        x = x1-x
        x1= x
        
        
        x = self.rem3_conv1(x)
        x = self.rem3_bn1(x)
        x = torch.cat((x,x1),dim=1)
        x = self.rem3_conv2(x)
        x = self.rem3_bn2(x)
        x = torch.cat((x,x1),dim=1)
        x = self.rem3_conv3(x)
        x = self.rem3_bn3(x)
        
        x = x1 - x
        
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.prelu1(x)
        
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.prelu2(x)
        
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.prelu3(x)
        
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.prelu4(x)
        
        x = self.avgpool1(x)
        
        x = self.conv5(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.softmax(x)
       
        return x

In [11]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0')
model = Net().to(device)

In [12]:
model = torch.load("Rafi_trained_model_similar_final")
model.eval()

results_folder = "Results_Random__"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Random__ folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

a=-1

with torch.no_grad():
    for batch_idx, (imgs1, labels1,patch_filename) in enumerate(test_loader_random):
        _, c = torch.max(labels1.data,1)
        d = c.cpu().numpy()[0]

        if(d!=a):
            print("Yes_Class",d)
            a= d
            z = d
            file_class = os.path.join(results_folder, f"Test_Class_{z}.csv")

            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Patch_Filename","True Class","Predicted Class","Probability of Predicted Class"])
    


        img_org,target = imgs1.to(device,dtype=torch.float), labels1.to(device)
       
        output = model(img_org)
        

        _, actual = torch.max(target.data, 1)    
        _, predicted = torch.max(output.data, 1)

        y_true = actual.cpu().numpy()[0]
        y_pred =predicted.cpu().numpy()[0]

        prob_y_pred = output[0][y_pred]
        prob_y_pred = prob_y_pred.cpu().numpy()
        prob_y_pred = np.around(prob_y_pred,decimals=2)

        if(batch_idx % 1000 == 0):
          print(batch_idx,patch_filename,y_true,y_pred,prob_y_pred)
        
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_idx,patch_filename,y_true,y_pred,prob_y_pred])   

Yes_Class 0
0 ('D01_rnd_130_3_7',) 0 0 0.85
1000 ('D01_rnd_35_17_3',) 0 0 0.64
2000 ('D01_rnd_43_18_5',) 0 0 0.94
3000 ('D01_rnd_124_19_11',) 0 0 0.79
4000 ('D01_rnd_135_6_4',) 0 29 0.71
5000 ('D01_rnd_44_9_5',) 0 13 0.44
6000 ('D01_rnd_43_8_12',) 0 26 0.81
7000 ('D01_rnd_113_6_9',) 0 0 0.37
8000 ('D01_rnd_166_10_4',) 0 0 0.99
9000 ('D01_rnd_113_15_2',) 0 0 0.97
10000 ('D01_rnd_88_19_1',) 0 0 0.97
Yes_Class 28
11000 ('D29_rnd_141_18_10',) 28 28 0.73
12000 ('D29_rnd_98_17_8',) 28 2 0.37
13000 ('D29_rnd_59_10_4',) 28 28 1.0
14000 ('D29_rnd_19_5_3',) 28 28 0.62
15000 ('D29_rnd_99_3_9',) 28 28 1.0
16000 ('D29_rnd_4_2_14',) 28 5 0.63
17000 ('D29_rnd_19_16_5',) 28 28 0.97
18000 ('D29_rnd_7_3_6',) 28 10 0.56
19000 ('D29_rnd_81_9_0',) 28 28 0.91
20000 ('D29_rnd_28_18_15',) 28 28 1.0
21000 ('D29_rnd_146_5_4',) 28 28 0.58
22000 ('D29_rnd_39_11_8',) 28 28 0.97
Yes_Class 22
23000 ('D23_rnd_10_2_11',) 22 26 0.3
24000 ('D23_rnd_99_6_14',) 22 24 0.49
25000 ('D23_rnd_18_19_14',) 22 23 0.48
26000 ('D23

211000 ('D24_rnd_141_12_12',) 23 23 0.85
212000 ('D24_rnd_33_8_5',) 23 27 0.65
Yes_Class 25
213000 ('D26_rnd_10_18_0',) 25 25 0.97
214000 ('D26_rnd_3_15_5',) 25 25 0.54
215000 ('D26_rnd_99_13_11',) 25 25 0.81
216000 ('D26_rnd_122_1_15',) 25 25 0.87
217000 ('D26_rnd_110_8_2',) 25 23 0.62
218000 ('D26_rnd_138_13_1',) 25 25 0.91
219000 ('D26_rnd_68_19_0',) 25 25 0.85
220000 ('D26_rnd_141_2_3',) 25 25 0.95
221000 ('D26_rnd_38_3_0',) 25 25 0.81
222000 ('D26_rnd_3_6_2',) 25 25 1.0
223000 ('D26_rnd_4_12_14',) 25 25 1.0
Yes_Class 9
224000 ('D10_rnd_41_17_9',) 9 9 1.0
225000 ('D10_rnd_83_16_6',) 9 9 1.0
226000 ('D10_rnd_113_4_5',) 9 9 1.0
227000 ('D10_rnd_85_2_1',) 9 9 1.0
228000 ('D10_rnd_150_6_8',) 9 9 1.0
229000 ('D10_rnd_32_1_1',) 9 9 1.0
230000 ('D10_rnd_129_18_11',) 9 9 1.0
231000 ('D10_rnd_153_9_12',) 9 9 1.0
232000 ('D10_rnd_53_10_2',) 9 9 1.0
233000 ('D10_rnd_33_15_13',) 9 9 1.0
Yes_Class 29
234000 ('D30_rnd_37_9_4',) 29 23 0.53
235000 ('D30_rnd_39_5_15',) 29 18 0.73
236000 ('D30_rnd_1

In [13]:
csv_dir = img_dir = "Results_Random__/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

In [14]:
a = "a"

results_folder = "Results_Clusters_Random__"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Clusters_Random__ folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

for f in files:
    print(f)
    b = "-1"
    d = Path(f).stem
    classname = Path(f).stem
    classname = int(classname.split("_")[2])
    
    true_image_class = classname
    
    if(d!=a):
        a = d
        file_class = os.path.join(results_folder, f"Test_Class_{a}.csv")
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Cluster_Name","True Class","Predicted Class","Probability of Predicted Class"])
    
    
    df = pd.read_csv(f)
    data = df.sort_values(by=['Patch_Filename'])
    data = data.to_numpy()
    
    predictions = []
    
    for i in range(len(data)):
        
        patchname = data[i][1]
        clustername =patchname.split("_")[:-1]
        clustername = "_".join(clustername)
        
        if(b!=clustername and b!="-1"):
            b = clustername
            
            pred_img_label  = max(predictions,key=predictions.count)
            prob = (predictions.count(pred_img_label))/len(predictions)
            pred_img_label = int(pred_img_label)
            
            
            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([i, clustername ,true_image_class,pred_img_label,prob])
            
            predictions = []
            predicted_label = data[i][3]
            predictions.append(predicted_label)
                
        elif(b!=clustername and b =="-1"):
            b = clustername
        
            predictions = []
            predicted_label = data[i][3]
            print(predicted_label)
            predictions.append(predicted_label)
            
        elif(b==clustername):
            predicted_label = data[i][3]
            predictions.append(predicted_label)
        else:
            print("Done")
    
    
    pred_img_label  = max(predictions,key=predictions.count)
    prob = (predictions.count(pred_img_label))/len(predictions)
    pred_img_label = int(pred_img_label)


    with open(file_class, 'a+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i, clustername ,true_image_class,pred_img_label,prob])
   
    predictions = []
    

Results_Random__/Test_Class_6.csv
6
Results_Random__/Test_Class_15.csv
15
Results_Random__/Test_Class_18.csv
14
Results_Random__/Test_Class_11.csv
17
Results_Random__/Test_Class_19.csv
10
Results_Random__/Test_Class_4.csv
4
Results_Random__/Test_Class_9.csv
9
Results_Random__/Test_Class_7.csv
7
Results_Random__/Test_Class_1.csv
1
Results_Random__/Test_Class_2.csv
2
Results_Random__/Test_Class_13.csv
13
Results_Random__/Test_Class_21.csv
21
Results_Random__/Test_Class_0.csv
20
Results_Random__/Test_Class_8.csv
8
Results_Random__/Test_Class_25.csv
23
Results_Random__/Test_Class_24.csv
29
Results_Random__/Test_Class_12.csv
13
Results_Random__/Test_Class_20.csv
21
Results_Random__/Test_Class_16.csv
7
Results_Random__/Test_Class_14.csv
14
Results_Random__/Test_Class_3.csv
3
Results_Random__/Test_Class_17.csv
17
Results_Random__/Test_Class_5.csv
5
Results_Random__/Test_Class_29.csv
29
Results_Random__/Test_Class_27.csv
16
Results_Random__/Test_Class_22.csv
3
Results_Random__/Test_Class_26.cs

In [15]:
csv_dir = img_dir = "Results_Clusters_Random__/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

In [16]:
with open(os.path.join(csv_dir, 'Image_Level_Results.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Class Label","Number of Images",\
                         "Correct Predicted Images",\
                         "Number of Cluster",\
                         "Total Clusters(Correct Classified Images)",\
                         "Correct Predicted Clusters(Correct Classified Images)",\
                         "Precetange Votes Per Image(Only Correct Images)",\
                         "Average Probability of Correct Patch(Only Correct Images)"])

In [17]:
for f in files:
    df = pd.read_csv(f)
    data = df.sort_values(by=['Cluster_Name'])
    classname = Path(f).stem
    classname = int(classname.split("_")[-1])
    a="a"

    true_image_class = classname
    total_images_perclass = 0
    correct_images_perclass = 0
    total_patches_perclass = 0

    total_class_votes =0
    total_class_patches =0
    prob_avg_correct_patch = 0.0
    votes = 0

    arr_pred_patches = []
    arr_pred_patches_prob = []
    total_correc_img_patches = 0


    for ind in data.index:
            filename = df['Cluster_Name'][ind]
            pred_patch_class = df['Predicted Class'][ind]
            pred_patch_prob = df['Probability of Predicted Class'][ind]
            
            filename = filename.split("_")
            file_length = len(filename)
            initial_filename = filename[:-1]
            patch_name = filename[file_length-1]
            #print(initial_filename)
            #print(patch_name)

            if(a!= initial_filename and a=="a"):
                a = initial_filename



            if(a!=initial_filename and a!="a"):
                total_images_perclass = total_images_perclass + 1

                counts = np.bincount(arr_pred_patches)
                pred_image_class = np.argmax(counts)
                votes = np.count_nonzero(arr_pred_patches==pred_image_class)
                s=0
                
                if(pred_image_class == true_image_class):
                    correct_images_perclass = correct_images_perclass + 1
                    total_class_votes = total_class_votes + votes
                    total_class_patches = total_class_patches + len(arr_pred_patches)
                    
                    total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

                    z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
                    for i in range(0,len(arr_pred_patches)):
                        if(z[i]==1):
                            s= s+1
                            prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]
            
                arr_pred_patches = []
                arr_pred_patches_prob = []
                a = initial_filename



            if(a==initial_filename):
                total_patches_perclass = total_patches_perclass + 1
                arr_pred_patches.append(pred_patch_class)
                arr_pred_patches_prob.append(pred_patch_prob)



    total_images_perclass = total_images_perclass + 1

    counts = np.bincount(arr_pred_patches)
    pred_image_class = np.argmax(counts)
    votes = np.count_nonzero(arr_pred_patches==pred_image_class)

    if(pred_image_class == true_image_class):
        correct_images_perclass = correct_images_perclass + 1
        total_class_votes = total_class_votes + votes
        total_class_patches = total_class_patches + len(arr_pred_patches)
        
        total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

        z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
        for i in range(0,len(arr_pred_patches)):
            if(z[i]==1):
                prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]

    arr_pred_patches = []
    arr_pred_patches_prob = []
    
    if(correct_images_perclass!=0):
        prob_avg_correct_patch = prob_avg_correct_patch / total_class_votes
        prob_avg_correct_patch = np.around(prob_avg_correct_patch,decimals=2)
        avg_vote_perclass = np.around((total_class_votes*100) / total_correc_img_patches,decimals=2)
        print(total_images_perclass,total_patches_perclass,correct_images_perclass,total_class_votes,avg_vote_perclass,prob_avg_correct_patch)  

        with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])
    else:
        with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])

df = pd.read_csv(os.path.join(csv_dir + 'Image_Level_Results.csv'))

ILA_random = sum(df['Correct Predicted Images']) / sum(df['Number of Images']) * 100

print(f"Image Level Accuracy: {ILA_random}%")

31 620 24 351 73.12 0.59
34 680 32 608 95.0 0.87
32 640 30 598 99.67 0.98
38 760 23 438 95.22 0.82
33 660 30 543 90.5 0.7
32 640 28 509 90.89 0.81
36 720 29 447 77.07 0.63
33 660 33 657 99.55 0.91
31 620 28 475 84.67 0.74
43 860 39 708 90.65 0.79
32 640 19 315 82.89 0.65
35 700 25 438 87.6 0.7
30 600 16 263 82.19 0.62
56 1120 40 715 89.26 0.79
33 660 26 434 83.46 0.68
31 620 28 542 96.79 0.94
34 680 33 596 90.3 0.75
51 1020 46 863 93.8 0.78
35 700 28 434 77.5 0.72
47 940 46 887 96.41 0.82
31 620 16 231 72.19 0.59
34 680 27 436 80.74 0.72
35 700 22 308 70.0 0.52
34 680 23 401 86.98 0.66
44 880 41 667 81.44 0.63
32 640 24 439 91.46 0.78
31 620 18 212 58.89 0.57
31 620 22 396 89.8 0.76
40 800 39 751 96.28 0.87
32 640 30 509 84.83 0.72
Image Level Accuracy: 80.765639589169%


In [18]:
model.eval()

results_folder = "Results_Similar__"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Similar__ folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

a=-1

with torch.no_grad():
    for batch_idx, (imgs1, labels1,patch_filename) in enumerate(test_loader_similar):
        _, c = torch.max(labels1.data,1)
        d = c.cpu().numpy()[0]

        if(d!=a):
            print("Yes_Class",d)
            a= d
            z = d
            file_class = os.path.join(results_folder, f"Test_Class_{z}.csv")

            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Patch_Filename","True Class","Predicted Class","Probability of Predicted Class"])
    


        img_org,target = imgs1.to(device,dtype=torch.float), labels1.to(device)
       
        output = model(img_org)
        

        _, actual = torch.max(target.data, 1)    
        _, predicted = torch.max(output.data, 1)

        y_true = actual.cpu().numpy()[0]
        y_pred =predicted.cpu().numpy()[0]

        prob_y_pred = output[0][y_pred]
        prob_y_pred = prob_y_pred.cpu().numpy()
        prob_y_pred = np.around(prob_y_pred,decimals=2)

        if(batch_idx % 1000 == 0):
          print(batch_idx,patch_filename,y_true,y_pred,prob_y_pred)
        
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_idx,patch_filename,y_true,y_pred,prob_y_pred])   

Yes_Class 0
0 ('D01_nat_1_18_14',) 0 0 0.94
1000 ('D01_nat_39_18_12',) 0 0 0.99
2000 ('D01_obj_25_3_11',) 0 0 0.81
3000 ('D01_nat_1_7_0',) 0 0 0.58
4000 ('D01_tex_2_17_4',) 0 0 0.84
5000 ('D01_tex_29_18_6',) 0 0 0.92
6000 ('D01_nat_30_17_10',) 0 8 0.56
7000 ('D01_nat_49_6_5',) 0 0 0.62
8000 ('D01_obj_1_11_15',) 0 0 0.57
9000 ('D01_tex_36_9_2',) 0 0 0.64
10000 ('D01_nat_36_17_14',) 0 0 0.59
Yes_Class 28
11000 ('D29_obj_36_7_0',) 28 28 0.97
12000 ('D29_nat_29_13_14',) 28 28 0.58
13000 ('D29_obj_36_16_11',) 28 28 0.49
14000 ('D29_obj_1_16_8',) 28 28 1.0
15000 ('D29_nat_36_11_4',) 28 28 0.99
16000 ('D29_nat_39_4_0',) 28 28 0.94
17000 ('D29_color_4_6_14',) 28 28 0.99
18000 ('D29_obj_29_16_4',) 28 28 1.0
19000 ('D29_obj_1_17_8',) 28 28 1.0
20000 ('D29_obj_17_0_5',) 28 28 0.99
Yes_Class 22
21000 ('D23_nat_36_15_12',) 22 22 0.9
22000 ('D23_obj_39_2_5',) 22 22 0.99
23000 ('D23_color_14_15_7',) 22 22 0.69
24000 ('D23_tex_29_5_10',) 22 22 0.49
25000 ('D23_obj_25_14_2',) 22 22 0.98
26000 ('D23_nat

213000 ('D30_obj_5_2_12',) 29 29 0.99
214000 ('D30_tex_17_7_0',) 29 29 0.98
215000 ('D30_nat_39_14_1',) 29 20 0.34
Yes_Class 7
216000 ('D08_obj_25_11_11',) 7 7 1.0
217000 ('D08_obj_17_5_15',) 7 7 1.0
218000 ('D08_nat_17_6_10',) 7 7 1.0
219000 ('D08_tex_30_9_14',) 7 7 0.99
220000 ('D08_tex_25_17_2',) 7 7 1.0
221000 ('D08_nat_30_5_2',) 7 7 0.99
222000 ('D08_tex_25_19_13',) 7 7 1.0
223000 ('D08_nat_45_8_4',) 7 7 0.93
224000 ('D08_nat_17_3_8',) 7 7 1.0
225000 ('D08_nat_25_9_0',) 7 7 0.49
Yes_Class 13
226000 ('D14_obj_30_19_2',) 13 13 0.93
227000 ('D14_nat_1_5_2',) 13 20 0.5
228000 ('D14_tex_17_13_4',) 13 13 0.96
229000 ('D14_obj_39_3_12',) 13 13 0.99
230000 ('D14_obj_5_4_10',) 13 13 0.95
231000 ('D14_color_14_19_10',) 13 22 0.7
232000 ('D14_nat_36_9_9',) 13 13 0.49
233000 ('D14_obj_5_2_13',) 13 13 0.96
234000 ('D14_obj_36_0_10',) 13 13 0.64
235000 ('D14_nat_17_3_2',) 13 13 0.86
Yes_Class 6
236000 ('D07_nat_2_18_3',) 6 6 0.76
237000 ('D07_obj_5_2_5',) 6 6 1.0
238000 ('D07_obj_29_3_9',) 6 6 

In [19]:
csv_dir = img_dir = "Results_Similar__/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

In [20]:
a = "a"

results_folder = "Results_Clusters_Similar__"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Clusters_Similar__ folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

for f in files:
    print(f)
    b = "-1"
    d = Path(f).stem
    classname = Path(f).stem
    classname = int(classname.split("_")[2])
    
    true_image_class = classname
    
    if(d!=a):
        a = d
        file_class = os.path.join(results_folder, f"Test_Class_{a}.csv")
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Cluster_Name","True Class","Predicted Class","Probability of Predicted Class"])
    
    
    df = pd.read_csv(f)
    data = df.sort_values(by=['Patch_Filename'])
    data = data.to_numpy()
    
    predictions = []
    
    for i in range(len(data)):
        
        patchname = data[i][1]
        clustername =patchname.split("_")[:-1]
        clustername = "_".join(clustername)
        
        if(b!=clustername and b!="-1"):
            b = clustername
            
            pred_img_label  = max(predictions,key=predictions.count)
            prob = (predictions.count(pred_img_label))/len(predictions)
            pred_img_label = int(pred_img_label)
            
            
            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([i, clustername ,true_image_class,pred_img_label,prob])
            
            predictions = []
            predicted_label = data[i][3]
            predictions.append(predicted_label)
                
        elif(b!=clustername and b =="-1"):
            b = clustername
        
            predictions = []
            predicted_label = data[i][3]
            print(predicted_label)
            predictions.append(predicted_label)
            
        elif(b==clustername):
            predicted_label = data[i][3]
            predictions.append(predicted_label)
        else:
            print("Done")
    
    
    pred_img_label  = max(predictions,key=predictions.count)
    prob = (predictions.count(pred_img_label))/len(predictions)
    pred_img_label = int(pred_img_label)


    with open(file_class, 'a+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i, clustername ,true_image_class,pred_img_label,prob])
   
    predictions = []
    

Results_Similar__/Test_Class_6.csv
6
Results_Similar__/Test_Class_15.csv
15
Results_Similar__/Test_Class_18.csv
18
Results_Similar__/Test_Class_11.csv
11
Results_Similar__/Test_Class_19.csv
19
Results_Similar__/Test_Class_4.csv
4
Results_Similar__/Test_Class_9.csv
9
Results_Similar__/Test_Class_7.csv
21
Results_Similar__/Test_Class_1.csv
4
Results_Similar__/Test_Class_2.csv
10
Results_Similar__/Test_Class_13.csv
13
Results_Similar__/Test_Class_21.csv
21
Results_Similar__/Test_Class_0.csv
0
Results_Similar__/Test_Class_8.csv
8
Results_Similar__/Test_Class_25.csv
25
Results_Similar__/Test_Class_24.csv
24
Results_Similar__/Test_Class_12.csv
12
Results_Similar__/Test_Class_20.csv
23
Results_Similar__/Test_Class_16.csv
16
Results_Similar__/Test_Class_14.csv
15
Results_Similar__/Test_Class_3.csv
3
Results_Similar__/Test_Class_17.csv
23
Results_Similar__/Test_Class_5.csv
5
Results_Similar__/Test_Class_29.csv
15
Results_Similar__/Test_Class_27.csv
27
Results_Similar__/Test_Class_22.csv
18
Resu

In [21]:
csv_dir = img_dir = "Results_Clusters_Similar__/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

In [22]:
with open(os.path.join(csv_dir, 'Image_Level_Results.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Class Label","Number of Images",\
                         "Correct Predicted Images",\
                         "Number of Cluster",\
                         "Total Clusters(Correct Classified Images)",\
                         "Correct Predicted Clusters(Correct Classified Images)",\
                         "Precetange Votes Per Image(Only Correct Images)",\
                         "Average Probability of Correct Patch(Only Correct Images)"])

In [23]:
for f in files:
    df = pd.read_csv(f)
    data = df.sort_values(by=['Cluster_Name'])
    classname = Path(f).stem
    classname = int(classname.split("_")[-1])
    a="a"

    true_image_class = classname
    total_images_perclass = 0
    correct_images_perclass = 0
    total_patches_perclass = 0

    total_class_votes =0
    total_class_patches =0
    prob_avg_correct_patch = 0.0
    votes = 0

    arr_pred_patches = []
    arr_pred_patches_prob = []
    total_correc_img_patches = 0


    for ind in data.index:
            filename = df['Cluster_Name'][ind]
            pred_patch_class = df['Predicted Class'][ind]
            pred_patch_prob = df['Probability of Predicted Class'][ind]
            
            filename = filename.split("_")
            file_length = len(filename)
            initial_filename = filename[:-1]
            patch_name = filename[file_length-1]
            #print(initial_filename)
            #print(patch_name)

            if(a!= initial_filename and a=="a"):
                a = initial_filename



            if(a!=initial_filename and a!="a"):
                total_images_perclass = total_images_perclass + 1

                counts = np.bincount(arr_pred_patches)
                pred_image_class = np.argmax(counts)
                votes = np.count_nonzero(arr_pred_patches==pred_image_class)
                s=0
                
                if(pred_image_class == true_image_class):
                    correct_images_perclass = correct_images_perclass + 1
                    total_class_votes = total_class_votes + votes
                    total_class_patches = total_class_patches + len(arr_pred_patches)
                    
                    total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

                    z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
                    for i in range(0,len(arr_pred_patches)):
                        if(z[i]==1):
                            s= s+1
                            prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]
            
                arr_pred_patches = []
                arr_pred_patches_prob = []
                a = initial_filename



            if(a==initial_filename):
                total_patches_perclass = total_patches_perclass + 1
                arr_pred_patches.append(pred_patch_class)
                arr_pred_patches_prob.append(pred_patch_prob)



    total_images_perclass = total_images_perclass + 1

    counts = np.bincount(arr_pred_patches)
    pred_image_class = np.argmax(counts)
    votes = np.count_nonzero(arr_pred_patches==pred_image_class)

    if(pred_image_class == true_image_class):
        correct_images_perclass = correct_images_perclass + 1
        total_class_votes = total_class_votes + votes
        total_class_patches = total_class_patches + len(arr_pred_patches)
        
        total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

        z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
        for i in range(0,len(arr_pred_patches)):
            if(z[i]==1):
                prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]

    arr_pred_patches = []
    arr_pred_patches_prob = []
    
    if(correct_images_perclass!=0):
        prob_avg_correct_patch = prob_avg_correct_patch / total_class_votes
        prob_avg_correct_patch = np.around(prob_avg_correct_patch,decimals=2)
        avg_vote_perclass = np.around((total_class_votes*100) / total_correc_img_patches,decimals=2)
        print(total_images_perclass,total_patches_perclass,correct_images_perclass,total_class_votes,avg_vote_perclass,prob_avg_correct_patch)  

        with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])
    else:
        with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])

df = pd.read_csv(os.path.join(csv_dir + 'Image_Level_Results.csv'))

ILA_similar = sum(df['Correct Predicted Images']) / sum(df['Number of Images']) * 100

print(f"Image Level Accuracy: {ILA_similar}%")

32 640 26 414 79.62 0.74
32 640 30 596 99.33 0.96
32 640 32 640 100.0 1.0
32 640 30 556 92.67 0.9
32 640 32 576 90.0 0.76
32 640 31 599 96.61 0.87
32 640 29 524 90.34 0.85
32 640 32 626 97.81 0.94
32 640 32 617 96.41 0.88
32 640 32 605 94.53 0.89
32 640 30 578 96.33 0.86
32 640 32 613 95.78 0.83
32 640 30 565 94.17 0.84
32 640 30 593 98.83 0.91
32 640 32 602 94.06 0.79
32 640 31 612 98.71 0.92
32 640 31 611 98.55 0.92
32 640 32 611 95.47 0.9
32 640 32 633 98.91 0.9
32 640 32 632 98.75 0.9
32 640 32 631 98.59 0.84
32 640 31 592 95.48 0.81
32 640 31 570 91.94 0.79
32 640 28 525 93.75 0.87
32 640 28 525 93.75 0.84
32 640 32 632 98.75 0.91
32 640 30 559 93.17 0.82
32 640 30 519 86.64 0.82
32 640 32 627 97.97 0.93
32 640 30 566 94.33 0.86
Image Level Accuracy: 96.04166666666667%


In [24]:
model.eval()

results_folder = "Results_Merged__"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Merged__ folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

a=-1

with torch.no_grad():
    for batch_idx, (imgs1, labels1,patch_filename) in enumerate(test_loader_merged):
        _, c = torch.max(labels1.data,1)
        d = c.cpu().numpy()[0]

        if(d!=a):
            print("Yes_Class",d)
            a= d
            z = d
            file_class = os.path.join(results_folder, f"Test_Class_{z}.csv")

            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Patch_Filename","True Class","Predicted Class","Probability of Predicted Class"])
    


        img_org,target = imgs1.to(device,dtype=torch.float), labels1.to(device)
       
        output = model(img_org)
        

        _, actual = torch.max(target.data, 1)    
        _, predicted = torch.max(output.data, 1)

        y_true = actual.cpu().numpy()[0]
        y_pred =predicted.cpu().numpy()[0]

        prob_y_pred = output[0][y_pred]
        prob_y_pred = prob_y_pred.cpu().numpy()
        prob_y_pred = np.around(prob_y_pred,decimals=2)

        if(batch_idx % 1000 == 0):
          print(batch_idx,patch_filename,y_true,y_pred,prob_y_pred)
        
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_idx,patch_filename,y_true,y_pred,prob_y_pred])   

Yes_Class 0
0 ('D01_rnd_130_3_7',) 0 0 0.85
1000 ('D01_obj_25_12_1',) 0 3 0.65
2000 ('D01_rnd_88_16_3',) 0 0 0.71
3000 ('D01_rnd_150_5_1',) 0 17 0.68
4000 ('D01_rnd_86_0_10',) 0 20 0.93
5000 ('D01_rnd_84_1_12',) 0 15 0.44
6000 ('D01_tex_17_9_3',) 0 0 1.0
7000 ('D01_color_14_4_8',) 0 0 1.0
8000 ('D01_rnd_50_12_14',) 0 0 0.99
9000 ('D01_rnd_32_16_13',) 0 15 0.63
10000 ('D01_rnd_35_5_4',) 0 25 0.5
11000 ('D01_tex_30_10_15',) 0 10 0.4
12000 ('D01_rnd_124_14_0',) 0 1 0.62
13000 ('D01_tex_7_2_7',) 0 7 0.54
14000 ('D01_nat_29_18_3',) 0 20 0.25
15000 ('D01_rnd_124_17_0',) 0 0 0.69
16000 ('D01_rnd_141_1_5',) 0 13 0.6
17000 ('D01_rnd_84_14_5',) 0 0 0.98
18000 ('D01_rnd_150_13_8',) 0 16 0.31
19000 ('D01_rnd_146_6_5',) 0 2 0.75
20000 ('D01_tex_17_16_8',) 0 0 0.9
21000 ('D01_rnd_150_14_1',) 0 0 0.56
Yes_Class 28
22000 ('D29_color_4_14_0',) 28 28 0.93
23000 ('D29_rnd_7_4_14',) 28 28 0.89
24000 ('D29_tex_36_1_0',) 28 28 1.0
25000 ('D29_obj_30_4_4',) 28 28 0.99
26000 ('D29_obj_5_6_9',) 28 28 1.0
27000

215000 ('D18_rnd_7_10_11',) 17 17 0.68
216000 ('D18_rnd_130_16_2',) 17 21 0.44
217000 ('D18_obj_29_16_0',) 17 17 0.82
218000 ('D18_tex_7_18_12',) 17 17 0.78
219000 ('D18_rnd_124_2_11',) 17 17 0.51
220000 ('D18_tex_2_7_7',) 17 17 0.96
221000 ('D18_nat_30_2_11',) 17 17 1.0
222000 ('D18_rnd_5_6_5',) 17 17 0.88
223000 ('D18_rnd_2_12_11',) 17 14 0.34
Yes_Class 10
224000 ('D11_rnd_160_9_6',) 10 1 0.63
225000 ('D11_rnd_113_10_3',) 10 10 0.88
226000 ('D11_tex_36_15_15',) 10 10 0.99
227000 ('D11_rnd_166_3_4',) 10 20 0.86
228000 ('D11_obj_25_6_0',) 10 10 0.56
229000 ('D11_tex_1_17_14',) 10 10 0.74
230000 ('D11_tex_29_12_6',) 10 10 1.0
231000 ('D11_rnd_51_15_8',) 10 10 0.96
232000 ('D11_rnd_117_1_1',) 10 10 0.63
233000 ('D11_nat_1_4_9',) 10 10 0.66
234000 ('D11_nat_39_4_4',) 10 10 0.48
235000 ('D11_rnd_96_5_13',) 10 10 0.61
236000 ('D11_rnd_141_12_15',) 10 20 0.64
237000 ('D11_nat_36_4_14',) 10 10 0.99
238000 ('D11_rnd_122_8_3',) 10 23 0.3
239000 ('D11_color_2_9_13',) 10 10 0.27
240000 ('D11_colo

428000 ('D10_rnd_122_8_5',) 9 9 1.0
429000 ('D10_rnd_110_12_4',) 9 9 1.0
430000 ('D10_rnd_44_17_13',) 9 9 1.0
431000 ('D10_rnd_71_10_5',) 9 9 1.0
432000 ('D10_obj_1_19_14',) 9 9 1.0
433000 ('D10_tex_17_3_11',) 9 9 1.0
434000 ('D10_nat_17_5_0',) 9 9 1.0
435000 ('D10_tex_1_5_0',) 9 9 1.0
436000 ('D10_rnd_32_13_13',) 9 9 1.0
437000 ('D10_color_4_19_6',) 9 9 1.0
438000 ('D10_rnd_146_0_4',) 9 9 1.0
Yes_Class 29
439000 ('D30_rnd_135_2_9',) 29 23 0.5
440000 ('D30_rnd_138_4_2',) 29 29 0.92
441000 ('D30_rnd_58_11_10',) 29 1 0.46
442000 ('D30_rnd_124_12_3',) 29 29 0.61
443000 ('D30_tex_2_1_7',) 29 29 0.7
444000 ('D30_color_4_14_12',) 29 29 0.65
445000 ('D30_rnd_79_9_7',) 29 29 0.76
446000 ('D30_rnd_119_6_0',) 29 23 0.97
447000 ('D30_rnd_146_17_9',) 29 29 0.97
448000 ('D30_obj_39_8_7',) 29 29 0.96
449000 ('D30_rnd_85_2_13',) 29 11 0.45
450000 ('D30_nat_25_12_10',) 29 29 0.3
451000 ('D30_obj_29_5_15',) 29 29 0.47
452000 ('D30_obj_36_4_9',) 29 29 1.0
453000 ('D30_obj_29_2_11',) 29 29 0.94
454000 ('

640000 ('D09_rnd_138_12_2',) 8 8 0.99
641000 ('D09_nat_9_9_8',) 8 8 1.0
642000 ('D09_obj_39_16_15',) 8 8 1.0
643000 ('D09_obj_39_4_3',) 8 8 1.0
644000 ('D09_rnd_45_10_0',) 8 22 0.39
645000 ('D09_nat_45_17_14',) 8 8 1.0
646000 ('D09_nat_36_4_10',) 8 8 0.97
647000 ('D09_nat_17_16_15',) 8 8 0.89
648000 ('D09_rnd_45_10_13',) 8 25 0.39
649000 ('D09_obj_17_6_8',) 8 8 1.0


In [25]:
csv_dir = img_dir = "Results_Merged__/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

In [26]:
a = "a"

results_folder = "Results_Clusters_Merged__"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Clusters_Merged__ folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

for f in files:
    print(f)
    b = "-1"
    d = Path(f).stem
    classname = Path(f).stem
    classname = int(classname.split("_")[2])
    
    true_image_class = classname
    
    if(d!=a):
        a = d
        file_class = os.path.join(results_folder, f"Test_Class_{a}.csv")
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Cluster_Name","True Class","Predicted Class","Probability of Predicted Class"])
    
    
    df = pd.read_csv(f)
    data = df.sort_values(by=['Patch_Filename'])
    data = data.to_numpy()
    
    predictions = []
    
    for i in range(len(data)):
        
        patchname = data[i][1]
        clustername =patchname.split("_")[:-1]
        clustername = "_".join(clustername)
        
        if(b!=clustername and b!="-1"):
            b = clustername
            
            pred_img_label  = max(predictions,key=predictions.count)
            prob = (predictions.count(pred_img_label))/len(predictions)
            pred_img_label = int(pred_img_label)
            
            
            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([i, clustername ,true_image_class,pred_img_label,prob])
            
            predictions = []
            predicted_label = data[i][3]
            predictions.append(predicted_label)
                
        elif(b!=clustername and b =="-1"):
            b = clustername
        
            predictions = []
            predicted_label = data[i][3]
            print(predicted_label)
            predictions.append(predicted_label)
            
        elif(b==clustername):
            predicted_label = data[i][3]
            predictions.append(predicted_label)
        else:
            print("Done")
    
    
    pred_img_label  = max(predictions,key=predictions.count)
    prob = (predictions.count(pred_img_label))/len(predictions)
    pred_img_label = int(pred_img_label)


    with open(file_class, 'a+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i, clustername ,true_image_class,pred_img_label,prob])
   
    predictions = []
    

Results_Merged__/Test_Class_6.csv
6
Results_Merged__/Test_Class_15.csv
15
Results_Merged__/Test_Class_18.csv
18
Results_Merged__/Test_Class_11.csv
11
Results_Merged__/Test_Class_19.csv
19
Results_Merged__/Test_Class_4.csv
4
Results_Merged__/Test_Class_9.csv
9
Results_Merged__/Test_Class_7.csv
21
Results_Merged__/Test_Class_1.csv
4
Results_Merged__/Test_Class_2.csv
10
Results_Merged__/Test_Class_13.csv
13
Results_Merged__/Test_Class_21.csv
21
Results_Merged__/Test_Class_0.csv
0
Results_Merged__/Test_Class_8.csv
8
Results_Merged__/Test_Class_25.csv
25
Results_Merged__/Test_Class_24.csv
24
Results_Merged__/Test_Class_12.csv
12
Results_Merged__/Test_Class_20.csv
23
Results_Merged__/Test_Class_16.csv
16
Results_Merged__/Test_Class_14.csv
15
Results_Merged__/Test_Class_3.csv
3
Results_Merged__/Test_Class_17.csv
23
Results_Merged__/Test_Class_5.csv
5
Results_Merged__/Test_Class_29.csv
15
Results_Merged__/Test_Class_27.csv
27
Results_Merged__/Test_Class_22.csv
18
Results_Merged__/Test_Class_26

In [27]:
csv_dir = img_dir = "Results_Clusters_Merged__/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

In [28]:
with open(os.path.join(csv_dir, 'Image_Level_Results.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Class Label","Number of Images",\
                         "Correct Predicted Images",\
                         "Number of Cluster",\
                         "Total Clusters(Correct Classified Images)",\
                         "Correct Predicted Clusters(Correct Classified Images)",\
                         "Precetange Votes Per Image(Only Correct Images)",\
                         "Average Probability of Correct Patch(Only Correct Images)"])

In [29]:
for f in files:
    df = pd.read_csv(f)
    data = df.sort_values(by=['Cluster_Name'])
    classname = Path(f).stem
    classname = int(classname.split("_")[-1])
    a="a"

    true_image_class = classname
    total_images_perclass = 0
    correct_images_perclass = 0
    total_patches_perclass = 0

    total_class_votes =0
    total_class_patches =0
    prob_avg_correct_patch = 0.0
    votes = 0

    arr_pred_patches = []
    arr_pred_patches_prob = []
    total_correc_img_patches = 0


    for ind in data.index:
            filename = df['Cluster_Name'][ind]
            pred_patch_class = df['Predicted Class'][ind]
            pred_patch_prob = df['Probability of Predicted Class'][ind]
            
            filename = filename.split("_")
            file_length = len(filename)
            initial_filename = filename[:-1]
            patch_name = filename[file_length-1]
            #print(initial_filename)
            #print(patch_name)

            if(a!= initial_filename and a=="a"):
                a = initial_filename



            if(a!=initial_filename and a!="a"):
                total_images_perclass = total_images_perclass + 1

                counts = np.bincount(arr_pred_patches)
                pred_image_class = np.argmax(counts)
                votes = np.count_nonzero(arr_pred_patches==pred_image_class)
                s=0
                
                if(pred_image_class == true_image_class):
                    correct_images_perclass = correct_images_perclass + 1
                    total_class_votes = total_class_votes + votes
                    total_class_patches = total_class_patches + len(arr_pred_patches)
                    
                    total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

                    z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
                    for i in range(0,len(arr_pred_patches)):
                        if(z[i]==1):
                            s= s+1
                            prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]
            
                arr_pred_patches = []
                arr_pred_patches_prob = []
                a = initial_filename



            if(a==initial_filename):
                total_patches_perclass = total_patches_perclass + 1
                arr_pred_patches.append(pred_patch_class)
                arr_pred_patches_prob.append(pred_patch_prob)



    total_images_perclass = total_images_perclass + 1

    counts = np.bincount(arr_pred_patches)
    pred_image_class = np.argmax(counts)
    votes = np.count_nonzero(arr_pred_patches==pred_image_class)

    if(pred_image_class == true_image_class):
        correct_images_perclass = correct_images_perclass + 1
        total_class_votes = total_class_votes + votes
        total_class_patches = total_class_patches + len(arr_pred_patches)
        
        total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

        z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
        for i in range(0,len(arr_pred_patches)):
            if(z[i]==1):
                prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]

    arr_pred_patches = []
    arr_pred_patches_prob = []
    
    if(correct_images_perclass!=0):
        prob_avg_correct_patch = prob_avg_correct_patch / total_class_votes
        prob_avg_correct_patch = np.around(prob_avg_correct_patch,decimals=2)
        avg_vote_perclass = np.around((total_class_votes*100) / total_correc_img_patches,decimals=2)
        print(total_images_perclass,total_patches_perclass,correct_images_perclass,total_class_votes,avg_vote_perclass,prob_avg_correct_patch)  

        with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])
    else:
        with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])

df = pd.read_csv(os.path.join(csv_dir + 'Image_Level_Results.csv'))

ILA_merged = sum(df['Correct Predicted Images']) / sum(df['Number of Images']) * 100

print(f"Image Level Accuracy: {ILA_merged}%")

63 1260 50 765 76.5 0.67
66 1320 62 1204 97.1 0.92
64 1280 62 1238 99.84 0.99
70 1400 53 993 93.68 0.87
65 1300 62 1119 90.24 0.73
64 1280 59 1108 93.9 0.84
68 1360 58 971 83.71 0.75
65 1300 65 1283 98.69 0.92
63 1260 61 1101 90.25 0.81
75 1500 71 1312 92.39 0.83
64 1280 49 892 91.02 0.79
67 1340 57 1051 92.19 0.78
62 1240 46 827 89.89 0.77
88 1760 71 1313 92.46 0.85
65 1300 58 1036 89.31 0.75
63 1260 59 1154 97.8 0.93
66 1320 64 1207 94.3 0.83
83 1660 78 1474 94.49 0.83
67 1340 60 1067 88.92 0.83
79 1580 78 1519 97.37 0.85
63 1260 48 862 89.79 0.78
66 1320 58 1027 88.53 0.77
67 1340 53 878 82.83 0.7
66 1320 51 925 90.69 0.78
76 1520 69 1192 86.38 0.72
64 1280 56 1071 95.62 0.86
63 1260 48 772 80.42 0.75
63 1260 52 914 87.97 0.79
72 1440 71 1378 97.04 0.9
64 1280 60 1075 89.58 0.79
Image Level Accuracy: 88.0846873461349%


In [30]:
print(f"Image Level Accuracy (Trained on Similar Set, Tested on Random Set): {ILA_random}%")
print(f"Image Level Accuracy (Trained on Similar Set, Tested on Similar Set): {ILA_similar}%")
print(f"Image Level Accuracy (Trained on Similar Set, Tested on Merged Set): {ILA_merged}%")

Image Level Accuracy (Trained on Similar Set, Tested on Random Set): 80.765639589169%
Image Level Accuracy (Trained on Similar Set, Tested on Similar Set): 96.04166666666667%
Image Level Accuracy (Trained on Similar Set, Tested on Merged Set): 88.0846873461349%
